In [ ]:
!pip install --upgrade pip
# Installs the wheel compatible with CUDA 11 and cuDNN 8.2 or newer.
# Note: wheels only available on linux.
!pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

In [ ]:
import jax.numpy as jnp
from tensorflow import keras

## Data preparation

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [ ]:
print(f'x_train_shape: {x_train.shape} - y_train_shape: {y_train.shape}')
print(f'x_test_shape: {x_test.shape} - y_test_shape: {y_test.shape}')